In [1]:
import pandas as pd
import numpy as np

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'

In [3]:
!wget $data -O car_price.csv

--2023-10-04 21:32:13--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘car_price.csv’

car_price.csv       100%[===================>]   1.41M  --.-KB/s    in 0.02s   

2023-10-04 21:32:13 (64.8 MB/s) - ‘car_price.csv’ saved [1475504/1475504]



In [21]:
df = pd.read_csv('car_price.csv')
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


## Data preprocessing


In [22]:
df.columns

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP'],
      dtype='object')

In [23]:
base = ['Make','Model','Year','Engine Cylinders','Transmission Type','Vehicle Style','highway MPG','city mpg','MSRP']
base

['Make',
 'Model',
 'Year',
 'Engine Cylinders',
 'Transmission Type',
 'Vehicle Style',
 'highway MPG',
 'city mpg',
 'MSRP']

In [51]:
data = df[base]
data.rename(columns = {'MSRP':'price'}, inplace= True)

data.columns = data.columns.str.lower().str.replace(' ','_')


categorial_cols = list(data.dtypes[data.dtypes == 'object'].index)

for col in categorial_cols:
#     df[col] = df[col].str.lower().str.replace(' ', '_')
    pass
data = data.fillna(0)

data


/tmp/ipykernel_1234/4031705745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns = {'MSRP':'price'}, inplace= True)


,make,model,year,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,6.0,MANUAL,Convertible,28,18,34500
...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,6.0,AUTOMATIC,4dr Hatchback,23,16,46120
11910,Acura,ZDX,2012,6.0,AUTOMATIC,4dr Hatchback,23,16,56670
11911,Acura,ZDX,2012,6.0,AUTOMATIC,4dr Hatchback,23,16,50620
11912,Acura,ZDX,2013,6.0,AUTOMATIC,4dr Hatchback,23,16,50920


In [52]:
from sklearn.model_selection import train_test_split

In [53]:
df_full_train, df_test = train_test_split(data, test_size=0.2, random_state=1)

In [54]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [55]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_full_train = df_full_train.reset_index(drop= True)

In [56]:
y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values

In [57]:
del df_train['price']
del df_val['price']
del df_test['price']

In [58]:
df_full_train['transmission_type'].value_counts()
df_full_train['transmission_type'].describe()

count          9531
unique            5
top       AUTOMATIC
freq           6589
Name: transmission_type, dtype: object

In [59]:
df_full_train.dtypes

make                  object
model                 object
year                   int64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
price                  int64
dtype: object

In [60]:
numerical = ['year','engine_cylinders', 'highway_mpg', 'city_mpg']

In [61]:
df_full_train[numerical].corrwith(df_full_train.highway_mpg)

year                0.255552
engine_cylinders   -0.607881
highway_mpg         1.000000
city_mpg            0.878715
dtype: float64

In [62]:
df_full_train[numerical].corrwith(df_full_train.city_mpg)

year                0.198259
engine_cylinders   -0.588176
highway_mpg         0.878715
city_mpg            1.000000
dtype: float64

In [63]:
from sklearn.metrics import mutual_info_score

In [67]:
above_average = (df_full_train['price'] > df_full_train.price.mean()).astype(int)
above_average.mean()

0.2712202287273109

In [68]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, above_average)

In [75]:
df_full_train[categorial_cols].apply(mutual_info_churn_score).sort_values(ascending = True)

transmission_type    0.019174
vehicle_style        0.082621
make                 0.237396
model                0.459324
dtype: float64

In [76]:
from sklearn.feature_extraction import DictVectorizer

In [79]:
train_dicts = df_train[categorial_cols + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)

In [80]:
X_train = dv.fit_transform(train_dicts)

In [82]:
dv.get_feature_names_out()

array(['city_mpg', 'engine_cylinders', 'highway_mpg', 'make=Acura',
       'make=Alfa Romeo', 'make=Aston Martin', 'make=Audi', 'make=BMW',
       'make=Bentley', 'make=Bugatti', 'make=Buick', 'make=Cadillac',
       'make=Chevrolet', 'make=Chrysler', 'make=Dodge', 'make=FIAT',
       'make=Ferrari', 'make=Ford', 'make=GMC', 'make=Genesis',
       'make=HUMMER', 'make=Honda', 'make=Hyundai', 'make=Infiniti',
       'make=Kia', 'make=Lamborghini', 'make=Land Rover', 'make=Lexus',
       'make=Lincoln', 'make=Lotus', 'make=Maserati', 'make=Maybach',
       'make=Mazda', 'make=McLaren', 'make=Mercedes-Benz',
       'make=Mitsubishi', 'make=Nissan', 'make=Oldsmobile',
       'make=Plymouth', 'make=Pontiac', 'make=Porsche',
       'make=Rolls-Royce', 'make=Saab', 'make=Scion', 'make=Spyker',
       'make=Subaru', 'make=Suzuki', 'make=Tesla', 'make=Toyota',
       'make=Volkswagen', 'make=Volvo', 'model=1 Series',
       'model=1 Series M', 'model=100', 'model=124 Spider',
       'model=190-